In [5]:
import pandas as pd
import numpy as np 

# 
these_data = pd.read_excel('Data\Fulldata.xlsx')
persona_data = pd.read_excel('Data\Personas.xlsx')
raw_questions = pd.DataFrame(None, columns=["Prompt", "Answer", "AllAnswers"])
for i in range(97):
    for j in these_data.index:
        if (these_data["These: Nr."][j] == i):
            question = "Skala:\n 1: 'stimme gar nicht zu'\n 2: 'Stimme nur teilweise zu'\n 3: 'neutral'\n 4: 'Stimme stark zu' \n 5: 'stimme vollkommen zu'\n Stimmst du der folgenden Aussage zu? '"+ these_data["These: These"][j] +"' Antworte auf einer Skala von 1 bis 5.\n Antwort:"
            raw_questions.loc[len(raw_questions.index)] = {"Prompt": question}
            break
persona_questions = pd.DataFrame(None, columns=["PartyId", "SourceId", "Prompt", "TheseId", "PartyPosition", "Answer", "FullAnswer"])
for index, these_row in these_data.iterrows():
    for index, persona_row in persona_data.iterrows():
        if(these_row['Partei: Nr.'] == persona_row['PartyId']):
            prompt ="Selbstbeschreibung: '" + persona_row['Persona']+ "' " + "Skala:\n 1: 'stimme gar nicht zu'\n 2: 'Stimme nur teilweise zu'\n 3: 'neutral'\n 4: 'Stimme stark zu' \n 5: 'stimme vollkommen zu'\n Stimmst du der folgenden Aussage zu? '" + these_row['These: These'] + "' Antworte auf einer Skala von 1 bis 5.\n Antwort:"
            persona_questions.loc[len(persona_questions.index)] = {"PartyId": persona_row['PartyId'], "SourceId": persona_row['PartyId'], "Prompt": prompt, "TheseId": these_row['These: Nr.'], "PartyPosition": these_row['Position: Position']}

        

In [2]:
def saveToSvg(questionaire, name):
    questionaire.to_csv('Results/' + name + '.csv')

In [18]:
# dfki api requests
import requests
import json
import re
import numpy as np

def doExperiment(questionaire, raw_run):
  url = "http://serv-3340.kl.dfki.de:5000"
  for index, row in questionaire.iterrows():
    body = {
      "inputs": row['Prompt'],
      "parameters": {
        "best_of": 1,
        "max_new_tokens": 20,
        "repetition_penalty": 1.03,
        "seed": None,
        "temperature": 0.6,
        "top_k": 20,
        "top_n_tokens": 5,
        "top_p": 0.95,
        "typical_p": 0.95,
      }
    }
    json_body = json.dumps(body)
    headers = {'Content-Type': 'application/json'}
    
    if raw_run: 
      all = []
      full = 10
      for i in range(10):
        response = requests.post(url, data=json_body, headers=headers)
        json_response = response.json()
        match = re.search(r'Antwort:\s*([1-5])', json_response[0]['generated_text'])
        if match:
          number = int(match.group(1))
          all.append(number)
        else:
          full -= 1
      print(index)
      questionaire.loc[index, "Answer"] =  np.mean(all)
      questionaire.loc[index, "AllAnswers"] = str(all)
    else: 
      response = requests.post(url, data=json_body, headers=headers)
      json_response = response.json()
      match = re.search(r'Antwort:\s*([1-5])', json_response[0]['generated_text'])
      split_string = json_response[0]['generated_text'].split("Antwort:")
      if len(split_string) > 1:
        fullAnswer = split_string[1].strip()
      else:
        fullAnswer = ""
      if match:
        number = int(match.group(1))
        questionaire.loc[index, "Answer"] =  number
        questionaire.loc[index, "FullAnswer"] = fullAnswer
      else:
        questionaire.loc[index, "Answer"] =  None
        questionaire.loc[index, "FullAnswer"] = fullAnswer
      if index % 200 == 0:
        print(f"Progress: {index}/5184")
  return questionaire

In [20]:
result = doExperiment(persona_questions, False)
saveToSvg(result, "TheBloke_leo-hessianai-70B-chat-AWQ_persona")

Progress: 0/5184
Progress: 200/5184
Progress: 400/5184
Progress: 600/5184
Progress: 800/5184
Progress: 1000/5184
Progress: 1200/5184
Progress: 1400/5184
Progress: 1600/5184
Progress: 1800/5184
Progress: 2000/5184
Progress: 2200/5184
Progress: 2400/5184
Progress: 2600/5184
Progress: 2800/5184
Progress: 3000/5184
Progress: 3200/5184
Progress: 3400/5184
Progress: 3600/5184
Progress: 3800/5184
Progress: 4000/5184
Progress: 4200/5184
Progress: 4400/5184
Progress: 4600/5184
Progress: 4800/5184
Progress: 5000/5184
